## IMPORT LIBRARIES


In [20]:
import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import backend as K

## GET THE DATA & SPILT THE DATA

In [21]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape, y_train.shape)

(60000, 28, 28) (60000,)


## PRE-PROCESS THE DATA

In [22]:
#Normalizing The Image
x_train=x_train.astype(np.float32)/255
x_test=x_test.astype(np.float32)/255
x_train.shape

(60000, 28, 28)

In [23]:
#Reshape The Image
x_train=np.expand_dims(x_train,-1)
x_test=np.expand_dims(x_test,-1)
x_train.shape

(60000, 28, 28, 1)

In [24]:
# convert class vectors to binary class matrices
y_train=keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

In [27]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

In [28]:
y_test

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

## CREATING A MODEL

In [25]:
batch_size = 128
num_classes = 10
epochs = 10

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 12, 12, 64)        0         
                                                                 
 flatten (Flatten)           (None, 9216)              0         
                                                                 
 dense (Dense)               (None, 256)               2359552   
                                                        

## TRAINING THE MODEL

In [29]:
hist = model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test, y_test))
print("The model has successfully trained")

Epoch 1/10
469/469 [==============================] - 97s 204ms/step - loss: 2.2872 - accuracy: 0.1502 - val_loss: 2.2626 - val_accuracy: 0.3023
Epoch 2/10
469/469 [==============================] - 90s 192ms/step - loss: 2.2461 - accuracy: 0.2683 - val_loss: 2.2124 - val_accuracy: 0.4921
Epoch 3/10
469/469 [==============================] - 91s 194ms/step - loss: 2.1919 - accuracy: 0.3907 - val_loss: 2.1414 - val_accuracy: 0.6454
Epoch 4/10
469/469 [==============================] - 95s 202ms/step - loss: 2.1124 - accuracy: 0.4983 - val_loss: 2.0366 - val_accuracy: 0.7176
Epoch 5/10
469/469 [==============================] - 94s 201ms/step - loss: 1.9963 - accuracy: 0.5727 - val_loss: 1.8836 - val_accuracy: 0.7688
Epoch 6/10
469/469 [==============================] - 92s 197ms/step - loss: 1.8275 - accuracy: 0.6339 - val_loss: 1.6685 - val_accuracy: 0.8026
Epoch 7/10
469/469 [==============================] - 92s 196ms/step - loss: 1.6136 - accuracy: 0.6689 - val_loss: 1.4064 - val_ac

## SAVING THE MODEL

In [30]:
model.save('HDC.h5')
print("Saving the model as HDC.h5")

Saving the model as HDC.h5


## TESTING THE MODEL

In [31]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.7952084541320801
Test accuracy: 0.8399999737739563


## GUI


In [32]:
from keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image
import numpy as np

In [33]:
model = load_model('HDC.h5')

def predict_digit(img):
    #resize image to 28x28 pixels
    img = img.resize((28,28))
    #convert rgb to grayscale
    img = img.convert('L')
    img = np.array(img)
    #reshaping to support our model input and normalizing
    img = img.reshape(1,28,28,1)
    img = img/255.0
    #predicting the class
    res = model.predict([img])[0]
    return np.argmax(res), max(res)

class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)

        self.x = self.y = 0

        # Creating elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg = "white", cursor="cross")
        self.label = tk.Label(self, text="Thinking..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text = "Recognise", command =         self.classify_handwriting) 
        self.button_clear = tk.Button(self, text = "Clear", command = self.clear_all)

        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=W, )
        self.label.grid(row=0, column=1,pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)

        #self.canvas.bind("<Motion>", self.start_pos)
        self.canvas.bind("<B1-Motion>", self.draw_lines)

    def clear_all(self):
        self.canvas.delete("all")

    def classify_handwriting(self):
        HWND = self.canvas.winfo_id() # get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND) # get the coordinate of the canvas
        im = ImageGrab.grab(rect)

        digit, acc = predict_digit(im)
        self.label.configure(text= str(digit)+', '+ str(int(acc*100))+'%')

    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r=8
        self.canvas.create_oval(self.x-r, self.y-r, self.x + r, self.y + r, fill='black')

app = App()
mainloop()

1/1 [==============================] - 0s 161ms/step
